<a href="https://colab.research.google.com/github/srv96/AI-ML-TensorFlow/blob/main/TPU_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
import tensorflow as tf

# Automatically detect the TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

# List all available logical devices
print("All devices: ", tf.config.list_logical_devices('TPU'))


All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [12]:
strategy = tf.distribute.TPUStrategy(resolver)

Note : model creation must be in strategy scope

we will define the function now but this code

wont run outside the scope

In [18]:
def create_model(input_shape,num_classes):
  i = tf.keras.layers.Input(shape=input_shape)

  x = tf.keras.layers.Conv2D(32, (3,3), activation='relu' , padding = 'same')(i)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(32, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Conv2D(64, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(64, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Conv2D(128, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(128, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

  model = tf.keras.Model(i,x)
  return model

In [14]:
cifar10 = tf.keras.datasets.cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train , X_test = X_train / 255.0 , X_test / 255.0
y_train , y_test = y_train.flatten() , y_test.flatten()

print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
X_train.shape :  (50000, 32, 32, 3)
X_test.shape :  (10000, 32, 32, 3)
y_train.shape :  (50000,)
y_test.shape :  (10000,)


In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [20]:
K = len(np.unique(y_train))
print("Number of classes : ", K)

with strategy.scope():
  model = create_model(X_train.shape[1:], K)
  model.compile(
      optimizer = 'adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['sparse_categorical_accuracy']
  )

  batch_size = 256

  train_dataset = train_dataset.shuffle(1000).batch(batch_size)
  test_dataset = test_dataset.batch(batch_size)

  model.fit(
      train_dataset,
      epochs = 10,
      validation_data = test_dataset
  )


Number of classes :  10
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


196/196 ━━━━━━━━━━━━━━━━━━━━ 32s 111ms/step - loss: 2.0306 - sparse_categorical_accuracy: 0.3993 - val_loss: 2.9291 - val_sparse_categorical_accuracy: 0.2080
Epoch 2/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - loss: 0.9756 - sparse_categorical_accuracy: 0.6522 - val_loss: 2.8220 - val_sparse_categorical_accuracy: 0.2592
Epoch 3/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 15s 73ms/step - loss: 0.7423 - sparse_categorical_accuracy: 0.7335 - val_loss: 0.9190 - val_sparse_categorical_accuracy: 0.7160
Epoch 4/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step - loss: 0.6114 - sparse_categorical_accuracy: 0.7908 - val_loss: 0.8635 - val_sparse_categorical_accuracy: 0.7328
Epoch 5/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - loss: 0.4832 - sparse_categorical_accuracy: 0.8273 - val_loss: 0.7593 - val_sparse_categorical_accuracy: 0.7648
Epoch 6/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - loss: 0.4372 - sparse_categorical_accuracy: 0.8380 - val_loss: 0.8229 - val_sparse_categorical_accuracy: 0.7544
Ep

In [24]:
# model.save('cifar10.h5')
tf.keras.models.save_model(model, 'cifar10.keras')

In [26]:
with strategy.scope():
  model = tf.keras.models.load_model('cifar10.keras')
  model.compile(
      optimizer = 'adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['sparse_categorical_accuracy']
  )
  model.evaluate(test_dataset)
  out = model.predict(X_test[:1])
  print(out)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 30 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 0.7842 - sparse_categorical_accuracy: 0.7890
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 930ms/step
[[1.4932425e-05 6.6006010e-08 2.4679044e-04 9.9647021e-01 1.8663927e-06
  1.4778193e-03 1.7876541e-03 1.4634673e-08 5.6113504e-07 1.0952578e-07]]
